In [1]:
from datetime import datetime

import numpy as np
import pandas as pd
import pytz
from database import Database
from excel_writer import ExcelWriter
from sqlalchemy import func, select
from utils import OperatorCost

In [2]:
db = Database()

select_bookings = (
    select(
        db.users_user.c.name.label("purchase_manager"),
        db.gwg_reservation.c.status,
        db.clients_operator.c.short_name.label("operator_name"),
        db.gwg_reservation.c.ref_id,
        db.gwg_reservation.c.res_id,
        db.gwg_reservation.c.operator_code,
        db.gwg_reservation.c.bkg_ref,
        db.mapping_hotel.c.external_name.label("hotel_name"),
        db.gwg_reservation.c.guest_name,
        db.gwg_reservation.c.sales_date,
        db.gwg_reservation.c.in_date,
        db.gwg_reservation.c.out_date,
        db.gwg_reservation.c.room_type,
        db.gwg_reservation.c.meal,
        db.gwg_reservation.c.adult,
        db.gwg_reservation.c.child,
        db.gwg_reservation.c.days,
        db.gwg_reservation.c.purchase_price,
        db.gwg_reservation.c.sales_price,
        db.gwg_reservation.c.purchase_contract_id,
        db.gwg_reservation.c.purchase_spo_id,
        db.gwg_reservation.c.purchase_spo_code.label("purchase_code"),
        db.gwg_reservation.c.sales_contract_id,
        db.gwg_reservation.c.sales_spo_id,
        db.gwg_reservation.c.sales_spo_code.label("sales_code"),
        func.count()
        .over(
            partition_by=(
                db.gwg_reservation.c.bkg_ref,
                db.gwg_reservation.c.operator_code,
            )
        )
        .label("count"),
    )
    .outerjoin(
        db.mapping_hotel,
        db.gwg_reservation.c.hotel_id == db.mapping_hotel.c.external_code,
    )
    .outerjoin(
        db.mapping_operator,
        db.gwg_reservation.c.operator_id == db.mapping_operator.c.external_code,
    )
    .outerjoin(db.accommodation_hotel)
    .outerjoin(db.accommodation_purchase_manager)
    .outerjoin(db.users_user)
    .outerjoin(db.definitions_area)
    .outerjoin(db.definitions_region)
    .outerjoin(db.clients_operator)
    .where(
        func.date(db.gwg_reservation.c.create_date) == current_date.date(),
        db.gwg_reservation.c.status != "Can",
        db.clients_operator.c.category == "IC",
        db.mapping_operator.c.external_code != 1107,
    )
    .order_by(
        db.mapping_hotel.c.external_name,
        db.gwg_reservation.c.in_date,
        db.gwg_reservation.c.ref_id,
    )
)

/home/jovyan/work/database.py:34: SAWarning: Did not recognize type 'geometry' of column 'geom'
  Table(


NameError: name 'current_date' is not defined

In [4]:
local_tz = pytz.timezone("Asia/Dubai")
current_date = datetime.now(local_tz)

operator_cost = OperatorCost()

In [5]:
export = []

for destination in ["AE", "OM", "SA"]:
    df = pd.read_sql(
        sql=select_bookings.where(db.definitions_region.c.country_code == destination),
        con=db.engine.connect(),
    )

    bookings = operator_cost.set(df)

    outfile = f'GWG_{destination}_RESLIST_{datetime.now().strftime("%Y_%m_%d")}.xlsx'

    excel_writer = ExcelWriter([bookings], outfile)
    excel_writer.write_to_excel(number_column=[17, 18, 19], date_column=[9, 10, 11])
    excel_writer.write_diff_column(bookings)
    excel_writer.write_margin_column(bookings)
    excel_writer.set_conditional_format(bookings)
    excel_writer.writer.close()

    export.append(outfile)

In [6]:
from email_sender import EmailSender

sender = EmailSender(
    subject=f"Yesterday's Booking Costing - Test", to=["ivan.orara@meetingpointuae.com"]
)

sender.send_email(export)

In [ ]:
from datetime import datetime

import numpy as np
import pandas as pd
import pytz
from database import Database
from email_sender import EmailSender
from excel_writer import ExcelWriter
from sqlalchemy import func, select
from utils import OperatorCost